<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/semantic_vector6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=4baeb7dc0c755d2a583aabc961ce96b78f853a7839b18e05b10d002fee5d0f40
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import numpy as np
import re
import pandas as pd

In [83]:
# Read log templates file into a DataFrame
df = pd.read_csv('/content/HDFS_templates.csv')
df.head(3)

,EventId,EventTemplate
0,E1,<*>Adding an already existing block<*>
1,E2,<*>Verification succeeded for<*>
2,E3,<*>Served block<*>to<*>


In [84]:
stop_words =['a','about','above','after','again','against','ain','all','am','an','and','any','are','aren',"aren't",'as','at','be','because','been','before','being','below','between','both',
             'but','by','can','couldn',"couldn't",'d','did','didn',"didn't",'do','does','doesn',"doesn't",'doing','don',"don't",'during','each','few','for','from','further','had',
             'hadn',"hadn't",'has','hasn',"hasn't",'have','haven',"haven't",'having','he','her','here','hers','herself','him','himself','his','how','i','if','in','into','is','isn',
             "isn't",'it',"it's",'its','itself','just','ll','m','ma','me','mightn',"mightn't",'more','most','mustn',"mustn't",'my','myself','needn',"needn't",'nor','now','o','of',
             'once','only','or','other','our','ours','ourselves','out','own','re','s','same','shan',"shan't",'she',"she's",'should',"should've",'shouldn',"shouldn't",'so','some',
             'such','t','than','that',"that'll",'the','their','theirs','them','themselves','then','there','these','they','this','those','through','to','too','until','ve','very',
             'was','wasn',"wasn't",'we','were','weren',"weren't",'what','when','where','which','while','who','whom','why','will','with','won',"won't",'wouldn',"wouldn't",'y','you',
             "you'd","you'll","you're","you've",'your','yours','yourself','yourselves']

def tokenized(text):
  """
  Normalize text to extract most salient tokens
  """
  # replace special characters with space and remove digits
  text = re.sub(r'\W+', ' ', text) # replaces one or more non-alphanumeric characters (\W+) with a single space in the text
  text = re.sub('\d', '', text)    #  replaces any digit (\d) with an empty string in the text

  # convert camel case to snake case, then replace _ with space
  text = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', text)
  text = re.sub('([a-z0-9])([A-Z])', r'\1_\2', text).lower().replace('_', ' ')

  normalized_tokens = [w for w in text.split() if w not in stop_words]

  # Reconstruct the sentence
  filtered_sentence = ' '.join(normalized_tokens) # our output will be sentences not a list of words

  return filtered_sentence

In [85]:
tokenized_template = [tokenized(sentence) for sentence in df['EventTemplate'] ]
#normalized_templates[0] = ' '
print(tokenized_template)

['adding already existing block', 'verification succeeded', 'served block', 'got exception serving', 'receiving block src dest', 'received block src dest size', 'write block received exception', 'packet responder block interrupted', 'received block size', 'packet responder exception', 'packet responder block terminating', 'exception writing block mirror', 'receiving empty packet block', 'exception receive block block', 'changing block file offset block meta file offset', 'transmitted block', 'failed transfer got', 'starting thread transfer block', 'reopen block', 'unexpected error trying delete block block info not found volume map', 'deleting block file', 'block name system allocate block', 'block name system delete added invalid set', 'block removing block needed replications not belong file', 'block ask replicate', 'block name system add stored block block map updated added size', 'block name system add stored block redundant add stored block request received on size', 'block name s

# **PART1 : Using light sentence-transformers model**

In [7]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [86]:
templates2vec = {}

for i, sentence in enumerate(tokenized_template):
  templates2vec[i] = model.encode(sentence)

In [87]:
templates2vec[0]

array([-8.07234645e-02, -5.09827621e-02, -1.85776055e-02,  6.22143177e-03,
        3.87574895e-03,  2.78650727e-02, -4.20657098e-02, -1.74399931e-02,
        1.45915737e-02,  3.83097888e-03,  5.84656633e-02, -1.74974222e-02,
       -7.06486851e-02, -4.61918376e-02,  2.87673902e-02,  1.92356668e-02,
       -5.78267127e-02,  2.68602557e-02,  5.06063141e-02, -7.23769069e-02,
        6.43115258e-03,  2.89896931e-02, -4.41498272e-02,  1.85368787e-02,
        9.38612130e-03, -1.51463933e-02, -5.07418960e-02,  4.65034805e-02,
        8.56891349e-02, -7.92806968e-03,  6.13686070e-02, -1.37720993e-02,
        6.86893240e-02,  7.11067542e-02,  6.52557909e-02, -3.59213129e-02,
       -1.91655010e-02,  1.32184193e-01, -1.45765310e-02, -3.14246267e-02,
        2.17107404e-02,  8.83242209e-03,  2.75502889e-03,  5.26004750e-03,
       -5.67887947e-02, -2.02857088e-02, -2.88970191e-02,  2.89810766e-02,
        8.91672745e-02, -5.84360864e-03,  2.29576770e-02, -2.28244830e-02,
       -4.55667712e-02, -

In [88]:
def cosine_similarity(vector1, vector2):
    eps = 1e-8 #first template is zero marix , for avoiding zero in magnitudes
    # Calculate the dot product
    dot_product = np.dot(vector1, vector2)

    # Calculate the magnitudes
    magnitude1 = np.linalg.norm(vector1)+eps
    magnitude2 = np.linalg.norm(vector2)+eps

    # Calculate the cosine similarity
    cosine_similarity = dot_product / (magnitude1 * magnitude2)

    return cosine_similarity



In [89]:
tokenized_template[5]

'received block src dest size'

In [92]:
for i in range(30):
  print("Similarity:",cosine_similarity(templates2vec[0], templates2vec[i]))
  print(tokenized_template[0])
  print(tokenized_template[i])
  print('------------------------------------')

Similarity: 0.9999999800000006
adding already existing block
adding already existing block
------------------------------------
Similarity: 0.02131327571561643
adding already existing block
verification succeeded
------------------------------------
Similarity: 0.30955197788069205
adding already existing block
served block
------------------------------------
Similarity: 0.008829560885565937
adding already existing block
got exception serving
------------------------------------
Similarity: 0.342417471713052
adding already existing block
receiving block src dest
------------------------------------
Similarity: 0.2933779300188245
adding already existing block
received block src dest size
------------------------------------
Similarity: 0.3890350085175939
adding already existing block
write block received exception
------------------------------------
Similarity: 0.22932868737280737
adding already existing block
packet responder block interrupted
------------------------------------
Simi

In [96]:
for i in range(len(tokenized_template)):
  for j in range(i+1,len(tokenized_template)) :

      similarity = cosine_similarity(templates2vec[i], templates2vec[j])

      if similarity > 0.75 :
        print('similarity = ',similarity)
        print('template',i,' : ',tokenized_template[i])
        print('template',j,' : ',tokenized_template[j])
        print('--------------------------------')

similarity =  0.7855657181982927
template 4  :  receiving block src dest
template 5  :  received block src dest size
--------------------------------
similarity =  0.8161171511504964
template 5  :  received block src dest size
template 8  :  received block size
--------------------------------
similarity =  0.8719006006383125
template 7  :  packet responder block interrupted
template 10  :  packet responder block terminating
--------------------------------
similarity =  0.7676004475376894
template 25  :  block name system add stored block block map updated added size
template 26  :  block name system add stored block redundant add stored block request received on size
--------------------------------
similarity =  0.8484185168336622
template 26  :  block name system add stored block redundant add stored block request received on size
template 27  :  block name system add stored block add stored block request received on size not belong file
--------------------------------


PART2 : using word2vec and tff-idf

In [49]:
import gensim.downloader

In [50]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [51]:
# Download the 'word2vec-google-news-300' embeddings
word2vec = gensim.downloader.load('word2vec-google-news-300')

[=================================================-] 99.9% 1661.5/1662.8MB downloaded


In [53]:
word2vec.most_similar('add')

[('Adding', 0.578197181224823),
 ('Add', 0.5681695938110352),
 ('adds', 0.5657491683959961),
 ('bring', 0.5600515604019165),
 ('augment', 0.5254836678504944),
 ('create', 0.521090567111969),
 ('incorporate', 0.504277765750885),
 ('expand', 0.49926939606666565),
 ('combine', 0.49364253878593445),
 ('elevate', 0.4872509241104126)]

In [97]:
# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Compute TF-IDF features
tfidf_features = tfidf_vectorizer.fit_transform(tokenized_template)

# Access the TF-IDF feature matrix
print(tfidf_features.toarray().shape)

(30, 66)


In [98]:
dic = tfidf_vectorizer.vocabulary_
matrix_weight = tfidf_features.toarray()

templates2vec = {}

for i, sentence in enumerate(tokenized_template):
  vector = np.zeros(300) # same shape of embedding word : model.encode(word).shape
  for word in sentence.split():
    try:  # we define this for an empty template which word2vec.get_vector(word) would arise error
      j = dic[word]
      vector += matrix_weight[i,j] * word2vec.get_vector(word)
    except Exception:
      print(word) # print words without proper vector
      continue
  templates2vec[i] = vector

In [99]:
tokenized_template[1]

'verification succeeded'

In [100]:
matrix_weight[1] # ignored the word "bglerr"

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.70710678, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.70710678, 0.        , 0.        ,
       0.        ])

In [101]:
for i in range(30):
  print("Similarity:", cosine_similarity(templates2vec[0],templates2vec[i]))
  print(tokenized_template[0])
  print(tokenized_template[i])
  print('------------------------------------')

Similarity: 0.9999999918118969
adding already existing block
adding already existing block
------------------------------------
Similarity: 0.2192506647702711
adding already existing block
verification succeeded
------------------------------------
Similarity: 0.15047153507552957
adding already existing block
served block
------------------------------------
Similarity: 0.24609330566921747
adding already existing block
got exception serving
------------------------------------
Similarity: 0.1598574071986804
adding already existing block
receiving block src dest
------------------------------------
Similarity: 0.17812265527536378
adding already existing block
received block src dest size
------------------------------------
Similarity: 0.24656939420847437
adding already existing block
write block received exception
------------------------------------
Similarity: 0.10653356702749443
adding already existing block
packet responder block interrupted
------------------------------------
Sim

In [102]:
for i in range(len(tokenized_template)):
  for j in range(i+1,len(tokenized_template)) :

      similarity = cosine_similarity(templates2vec[i], templates2vec[j])

      if similarity > 0.75 :
        print('similarity = ',similarity)
        print('template',i,' : ',tokenized_template[i])
        print('template',j,' : ',tokenized_template[j])
        print('--------------------------------')

similarity =  0.8470709073132892
template 4  :  receiving block src dest
template 5  :  received block src dest size
--------------------------------
similarity =  0.7563605088634178
template 7  :  packet responder block interrupted
template 9  :  packet responder exception
--------------------------------
similarity =  0.7578512660012406
template 9  :  packet responder exception
template 10  :  packet responder block terminating
--------------------------------
similarity =  0.7589474517088791
template 21  :  block name system allocate block
template 25  :  block name system add stored block block map updated added size
--------------------------------
similarity =  0.824460417118458
template 25  :  block name system add stored block block map updated added size
template 26  :  block name system add stored block redundant add stored block request received on size
--------------------------------
similarity =  0.8301505325241675
template 25  :  block name system add stored block block 